In [ ]:
import pathlib

import pandas as pd
from IPython.display import display
import yaml

import dlsproc.xml
import dlsproc.bundle
import dlsproc.hier
import dlsproc.io
import dlsproc.parse
import dlsproc.postprocess
import dlsproc.assemble

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Outsiders

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

A file to *cache* the data read

In [ ]:
outsiders_output_file = dlsproc.io.File('outsiders')

The directory for *outsiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/dlsproc/data/agregados


The file belown was downloaded [here](https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/LicitacionesAgregacion.aspx). One could use something like
```
wget https://contrataciondelsectorpublico.gob.es/sindicacion/sindicacion_1044/PlataformasAgregadasSinMenores_2018.zip
```

In [ ]:
outsiders_zip_file = outsiders_directory / 'PlataformasAgregadasSinMenores_2021.zip'
assert outsiders_zip_file.exists()

The data is read from a file if present, or from the given **zip** if not

In [ ]:
if outsiders_output_file.exists():
    outsiders_df = outsiders_output_file.read()
else:
    # data is read from the given zip, and concatenated into a single `pd.DataFrame`...
    outsiders_df = dlsproc.bundle.read_zip(outsiders_zip_file, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_df)
    
    # ...and saved
    outsiders_output_file.write(outsiders_df)

How many entries and how many fields?

In [ ]:
outsiders_df.shape

(208204, 40)

Examples of columns

In [ ]:
outsiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RequiredCommodityClassification', 'ItemClassificationCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RealizedLocation', 'CountrySubentityCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'PlannedPeriod', 'DurationMeasure', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'ProcedureCode', '', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndDate', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndTime', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNotice

The first few rows

In [ ]:
outsiders_df.head()

id  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      https://contrataciondelestado.es/sindicacion/P...   
                                    1      https://contrataciondelestado.es/sindicacion/P...   
                                    2      https://contrataciondelestado.es/sindicacion/P...   
                                    3      https://contrataciondelestado.es/sindicacion/P...   
                                    4      https://contrataciondelestado.es/sindicacion/P...   

                                                                                     summary  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Id licitación: 3064_23/2021; Órgano de Contrat...   
                                    1      Id licitación: 3051_CONT_SERVICIOS/2021/40; Ór...   
                                    2      Id licitación: 3091_21/2021; Órgano de Contrat...   
                                    3      Id licitación: 3069_1/2021; Órgano de Contrata...   
                                    4      Id licitación: 8044_30146-0-21; Órgano de Cont...   

                                                                                       title  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Contrato del Servicio para el desarrollo de la...   
                                    1      REDACCIÓN DEL PLAN ESTRATÉGICO DE ACCESIBILIDA...   
                                    2      Contratación de obra de línea subterránea en A...   
                                    3      Construcción de depósito regulador de 100m3 en...   
                                    4          Renovación de la cubierta de edificio docente   

                                                                   updated  \
                                                                             
                                                                             
                                                                             
                               

Not every column is fo `str` type

In [ ]:
outsiders_df.dtypes[:20]

id                                                                                                                                        string
summary                                                                                                                                   string
title                                                                                                                                     string
updated                                                                                                                      datetime64[ns, UTC]
ContractFolderStatus  ContractFolderID                                                                                                    string
                      ContractFolderStatusCode                                                                                            string
                      LocatedContractingParty   Party                            PartyName                       Name             

There are *multivalued* columns (only showing 2).

In [ ]:
multivalued_columns = dlsproc.structure.multivalued_columns(outsiders_df)
multivalued_columns[:2]

[('ContractFolderStatus',
  'ProcurementProject',
  'RequiredCommodityClassification',
  'ItemClassificationCode',
  '',
  '',
  ''),
 ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', '')]

Inside them, sometimes valures are `list`s

In [ ]:
outsiders_df[multivalued_columns[:2]]

ContractFolderStatus  \
                                                                                ProcurementProject   
                                                                   RequiredCommodityClassification   
                                                                            ItemClassificationCode   
                                                                                                     
                                                                                                     
                                                                                                     
file name                                          entry                                             
PlataformasAgregadasSinMenores.atom                0                                      77310000   
                                                   1                                      71241000   
                                                   2                                      45231400   
                                                   3      [45231300, 45213260, 45112000, 44611600]   
                                                   4                                      45200000   
..                                                                                             ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463                                    45210000   
                                                   464                                    22200000   
                                                   465                                    80580000   
                                                   466                                    30200000   
                                                   467                                    71311210   

                                                                                 
                                                                ValidNoticeInfo  
                                                                 NoticeTypeCode  
                                                                                 
                                                                                 
                                                                                 
                                                                                 
file name                                          entry                         
PlataformasAgregadasSinMenores.atom                0                     DOC_CN  
                                                   1                     DOC_CN  
                                                   2                     DOC_CN  
                                                   3                     DOC_CN  
                                                   4                     DOC_CN  
..                                                                          ...  
PlataformasAgregadasSinMenores_20211231_030012_... 463                   DOC_CN  
                                                   464                   DOC_CN  
                                                   465                   DOC_CN  
                                                   466    [DOC_CN, DOC_CAN_ADJ]  
                                                   467                   DOC_CN  

[208204 rows x 2 columns]

### Entity

For the sake of convenience, *multiindex* column names are built for

- *Entity*

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '')

- *Amount*

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TaxExclusiveAmount',
 '',
 '',
 '')

- *Winner*

In [ ]:
winner_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'TenderResult', 'WinningParty', 'PartyName', 'Name'])
winner_col

('ContractFolderStatus',
 'TenderResult',
 'WinningParty',
 'PartyName',
 'Name',
 '',
 '')

Number of projects per entity along with the *accumulated* amount of money

In [ ]:
number_amount_df = outsiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
AGENCIA NAVARRA DE AUTONOMÍA Y DESARROLLO DE L...,1.040800e+07,8
AGENCIA NAVARRA PARA LA AUTONOMIA Y DESARROLLO...,1.796660e+06,2
AJUNTAMENT DE CERVIÀ DE TER,2.198030e+05,4
AJUNTAMENT DE FALSET,2.123535e+06,9
AJUNTAMENT DE SANT JAUME DELS DOMENYS,2.495736e+07,25
...,...,...
presidencia,3.065700e+05,6
Àrea Metropolitana de Barcelona,9.364749e+09,1410
ÁREA DE SALUD I (MURCIA OESTE - H.U. VIRGEN DE...,4.743750e+04,1


Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
Servicio Andaluz de Salud,2.757164e+10,3316
Consejería de Vivienda y Administración Local,2.476717e+10,38
"Canal de Isabel II, S.A.",1.622211e+10,1850
"Empresa Pública de Metro de Madrid, S.A.",1.288923e+10,3323
Ajuntament de Barcelona,1.030976e+10,2380
...,...,...
"ZUGAZTEL, M. P., S. A.",0.000000e+00,0
Directora Gerente Agencia Navarra de Autonomía...,0.000000e+00,0
"ITELAZPI, S.A.",0.000000e+00,0


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents

In [ ]:
docref_columns = dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')
docref_columns

MultiIndex([('ContractFolderStatus',            'ValidNoticeInfo', ...),
            ('ContractFolderStatus',     'LegalDocumentReference', ...),
            ('ContractFolderStatus',     'LegalDocumentReference', ...),
            ('ContractFolderStatus', 'TechnicalDocumentReference', ...),
            ('ContractFolderStatus', 'TechnicalDocumentReference', ...)],
           )

Columns containing *DocumentReference* in the name, allegedly URLs to documents. For a clearer visualization, `file name` and `zip` levels are dropped from the rows `pd.MultiIndex`.

In [ ]:
# the list insisde `droplevel` is extended with either *zip* or an empty list
outsiders_df.loc[:, docref_columns].droplevel(['file name'] + ['zip'] if 'zip' in outsiders_df.index.names else [])

ContractFolderStatus  \
                                                                                            ValidNoticeInfo   
                                                                                AdditionalPublicationStatus   
                                                                     AdditionalPublicationDocumentReference   
                                                                                                  IssueDate   
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores.atom                0                                             2021-12-31   
                                                   1                                             2022-01-01   
                                                   2                               [2021-12-17, 2021-12-31]   
                                                   3      [2021-12-16, 2021-12-16, 2021-12-16, 2021-12-1...   
                                                   4       [2021-11-24, 2021-12-10, 2021-12-16, 2021-12-31]   
..                                                                                                      ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463                                           2021-11-26   
                                                   464                                           2021-11-25   
                                                   465                             [2021-11-29, 2021-11-30]   
                                                   466                 [2021-11-19, 2021-11-22, 2021-12-29]   
                                                   467                             [2021-12-07, 2021-12-21]   

                                                                                                             \
                                                                                     LegalDocumentReference   
                                                                                                         ID   
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores.atom                0                                                   <NA>   
                                                   1                                                   <NA>   
                                                   2                                                   <NA>   
                                                   3                                                   <NA>   
                                                   4                                                   <NA>   
..                                                                                                      ...   
PlataformasAgregadasSinMenores_20211231_030012_... 463    2021.63 PCAP Reforma interior sede IRUÑA EITB.pdf   
                                                   464                2021.67 PCAP Suministro de Prensa.pdf   
                                                   465                                            PCAP.docx   
                                                   466                    3- PCP - Licenci

### Duplicates

In [ ]:
only_last_update_df = dlsproc.postprocess.keep_updates_only(outsiders_df)
only_last_update_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    https://contrataciondelestado.es/sindicacion/P...   
                                                   291    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   291    Id licitación: 2006OB07; Órgano de Contratació...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210101_030039.... 294    SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                                                   291    Subministrament d'energia elèctrica en BAIXA T...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

Size of the above `pd.DataFrame`

In [ ]:
only_last_update_df.shape

(64020, 40)

For the original `pd.DataFrame`

In [ ]:
outsiders_df.shape

(208204, 40)

A `pd.Series` yielding entries that were deleted and when (indexed by *file name* and `entry`)

In [ ]:
outsiders_deleted_series = dlsproc.bundle.read_deleted_zip(outsiders_zip_file)
outsiders_deleted_series

file name                                              id                                                                                 
PlataformasAgregadasSinMenores.atom                    https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8231713    2022-01-01 00:12:28.323000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8063576    2022-01-01 00:12:28.255000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/7824334    2022-01-01 00:12:28.189000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8163656    2022-01-01 00:12:28.143000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6834832    2022-01-01 00:12:28

Information about deleted entries is added to `only_last_update_df`

In [ ]:
stateful_outsiders_df = dlsproc.assemble.merge_deleted(only_last_update_df, outsiders_deleted_series)

The new *merged* `pd.DataFrame` has the same number of rows as the original one

In [ ]:
assert len(stateful_outsiders_df) == len(only_last_update_df)

How many entries have been deleted?

In [ ]:
stateful_outsiders_df['deleted_on'].notna().sum()

15

Some examples

In [ ]:
stateful_outsiders_df[stateful_outsiders_df['deleted_on'].notna()].head()

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20210108_030012.... https://contrataciondelestado.es/sindicacion/Pl...   342   
PlataformasAgregadasSinMenores_20210302_030022.... https://contrataciondelestado.es/sindicacion/Pl...   379   
PlataformasAgregadasSinMenores_20210306_030011_... https://contrataciondelestado.es/sindicacion/Pl...   217   
PlataformasAgregadasSinMenores_20210422_030025.... https://contrataciondelestado.es/sindicacion/Pl...   128   
PlataformasAgregadasSinMenores_20210705_030042.... https://contrataciondelestado.es/sindicacion/Pl...   380   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210108_030012.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 202100053;  Órgano de Contratac...   
PlataformasAgregadasSinMenores_20210302_030022.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 1/2021; Órgano de Contratación:...   
PlataformasAgregadasSinMenores_20210306_030011_... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 0032; Órgano de Contratación: F...   
PlataformasAgregadasSinMenores_20210422_030025.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2020182; Órgano de Contratación...   
PlataformasAgregadasSinMenores_20210705_030042.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: CONT1500033; Órgano de Contrata...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
     

### Completeness

Another column of interest: *URL entity*

In [ ]:
url_entity_col = dlsproc.hier.pad_col_levels(only_last_update_df, ['ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', ''])
url_entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'BuyerProfileURIID',
 '',
 '',
 '',
 '')

How many rows provide a non-null value for the entity URL?

In [ ]:
only_last_update_df[url_entity_col].notna().sum()

13755

What are the most repeated URLs?

In [ ]:
only_last_update_df.groupby(url_entity_col).size().sort_values()

(ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , , )
http://www.carm.es/neweb2/servlet/integra.serv...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
                                                     ... 
https://www.larioja.org/contratacion-publica/e...     275
http://www.madrid.org/cs/Satellite?cid=1205761...     277
https://contractaciopublica.gencat.cat/ecofin_...     343
https://contractaciopublica.gencat.cat/ecofin_...     364
http://www.madrid.org/cs/Satellite?cid=1204201...    1629
Length: 1203, dtype: int64

A new column with the (internet) domain is added

In [ ]:
only_last_update_df['domain'] = dlsproc.parse.domain(only_last_update_df)

In [ ]:
only_last_update_df['domain']

file name                                            entry
PlataformasAgregadasSinMenores_20210101_030039.atom  294      contratacion.euskadi.eus
                                                     291                          <NA>
                                                     288                          <NA>
                                                     282                          <NA>
                                                     281                          <NA>
                                                                        ...           
PlataformasAgregadasSinMenores.atom                  4                            <NA>
                                                     3                            <NA>
                                                     2                            <NA>
                                                     1                            <NA>
                                                     0                 

It is used for grouping

In [ ]:
domain_grouped_only_last_update_df = only_last_update_df.groupby('domain', dropna=False)

The following groups (*domain*s) are present, each one with the indicated number of entries

In [ ]:
domain_grouped_only_last_update_df.size()

domain
contractaciopublica.gencat.cat    12092
contratacion.euskadi.eus           3812
place-agregacion.carm.es             32
www.asturias.es                    1739
www.bilbao.eus                      171
www.carm.es                           1
www.contratacion.euskadi.eus       4786
www.contratosdegalicia.gal         1160
www.larioja.org                     275
www.madrid.org                     4787
NaN                               35165
dtype: int64

For the sake of convenience

In [ ]:
domain_col = dlsproc.hier.pad_col_levels(outsiders_df, ['domain'])
domain_col

('domain', '', '', '', '', '', '')

For every group we ascertain which elements (across every column) are not null (still a `pd.DataFrame`), and then summarize it (one value per group and column) with `mean` (ratio of filled-in values)

In [ ]:
filled_in_ratio_df = domain_grouped_only_last_update_df.apply(
    lambda x: x.notnull().mean()).drop(domain_col, axis=1)
    # lambda x: x.notnull().mean()).drop(dlsproc.hier.pad_col_levels(only_last_update_df, ['domain']), axis=1)
filled_in_ratio_df

id summary title updated  \
                                                            
                                                            
                                                            
                                                            
                                                            
                                                            
domain                                                      
contractaciopublica.gencat.cat  1.0     1.0   1.0     1.0   
contratacion.euskadi.eus        1.0     1.0   1.0     1.0   
place-agregacion.carm.es        1.0     1.0   1.0     1.0   
www.asturias.es                 1.0     1.0   1.0     1.0   
www.bilbao.eus                  1.0     1.0   1.0     1.0   
www.carm.es                     1.0     1.0   1.0     1.0   
www.contratacion.euskadi.eus    1.0     1.0   1.0     1.0   
www.contratosdegalicia.gal      1.0     1.0   1.0     1.0   
www.larioja.org                 1.0     1.0   1.0     1.0   
www.madrid.org                  1.0     1.0   1.0     1.0   
NaN                             1.0     1.0   1.0     1.0   

                               ContractFolderStatus                           \
                                   ContractFolderID ContractFolderStatusCode   
                                                                               
                                                                               
                                                                               
                                                                               
                                                                               
domain                                                                         
contractaciopublica.gencat.cat             0.999835                      1.0   
contratacion.euskadi.eus                   1.000000                      1.0   
place-agregacion.carm.es                   1.000000                      1.0   
www.asturias.es                            1.000000                      1.0   
www.bilbao.eus                             1.000000                      1.0   
www.carm.es                                1.000000                      1.0   
www.contratacion.euskadi.eus               1.000000                      1.0   
www.contratosdegalicia.gal                 1.000000                      1.0   
www.larioja.org                            1.000000                      1.0   
www.madrid.org                             1.000000                      1.0   
NaN                                        0.995507                      1.0   

                                                                           \
                               LocatedContractingParty                      
                                                 Party ParentLocatedParty   
                                             PartyName          PartyName   
                                                  Name               Name   
                                                                            
                                                                            
domain                                                                      
contractaciopublica.gencat.cat                     1.0           0.999917   
contratacion.euskadi.eus                           1.0           1.000000   
place-agregacion.carm.es                           1.0           1.000000   
www.asturias.es                                    1.0           0.000000   
www.bilbao.eus                                     1.0           1.000000   
www.carm.es                                        1.0           1.000000   
www.contratacion.euskadi.eus                       1.0           1.000000   
www.contratosdegalicia.gal                         1.0           0.000000   
www.larioja.org                                    1.0           1.000000   
www.madrid.org                             

More readable (notice the ordering of the index)

In [ ]:
tidy_filled_in_ratio_df = filled_in_ratio_df.sort_index(axis=1).T
tidy_filled_in_ratio_df

domain                                                                                                                                                        contractaciopublica.gencat.cat  \
ContractFolderStatus ContractFolderID                                                                                                                                               0.999835   
                     ContractFolderStatusCode                                                                                                                                       1.000000   
                     LegalDocumentReference     Attachment                          ExternalReference                      URI                                                      0.434916   
                                                ID                                                                                                                                  0.434916   
                     LocatedContractingParty    BuyerProfileURIID                                                                                                                   0.861148   
                                                ParentLocatedParty                  ParentLocatedParty                     ParentLocatedParty PartyName Name                        0.000000   
                                                                                                                           PartyName          Name                                  0.000000   
                                                                                    PartyName                              Name                                                     0.999917   
                                                Party                               PartyIdentification                    ID                                                       0.000083   
                                                                                    PartyName                              Name                                                     1.000000   
                     ProcurementProject         BudgetAmount                        EstimatedOverallContractAmount                                                                  1.000000   
                                                                                    TaxExclusiveAmount                                                                              1.000000   
                                                Name                                                                                                                                1.000000   
                                                PlannedPeriod                       DurationMeasure                                                                                 0.904152   
                                                                                    EndDate                                                                                         0.095270   
                                                                                    StartDate                                                                                       0.095270   
                                                RealizedLocation                    CountrySubentityCode                                                                            0.999090   
                                                RequiredCommodityClassification     ItemClassificationCode                                                                          0.988505   
                                                TypeCode                                                                                                                            1.000000   
                     TechnicalDocumentReference Attachment                          ExternalReference                      URI                                  

It is saved to a spreadsheet

In [ ]:
tidy_filled_in_ratio_df.to_excel('outsiders_filled_in.xlsx')

#### GenCat

For the sake of convenience

In [ ]:
id_col = dlsproc.hier.pad_col_levels(outsiders_df, ['id'])
id_col

('id', '', '', '', '', '', '')

Entries for *Generalitat de Catalunya*

In [ ]:
gencat_df = only_last_update_df[only_last_update_df['domain'] == 'contractaciopublica.gencat.cat']
gencat_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    https://contrataciondelestado.es/sindicacion/P...   
                                                   343    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Id licitación: 001/20001709; Órgano de Contrat...   
                                                   343    Id licitación: X2020025214; Órgano de Contrata...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Serveis de Manteniment i subministrament de le...   
                                                   343    És objecte de la venda, pel procediment obert ...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

How many entries from *GenCat*

In [ ]:
len(gencat_df)

12092

*GenCat* filling of both *Entity* and *WinningParty*. Notice the **data being explored are referred to the last update** meaning that a ratio of 0 for some `file name` and column only entails that the entries *that were updated for the last time in that file* (and only those) all have the corresponding column empty.

In [ ]:
filename_notna_ratio = gencat_df[[url_entity_col, winner_col]].groupby('file name').apply(lambda x: x.notna().mean())
filename_notna_ratio

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20210105_030039....                     0.0   
PlataformasAgregadasSinMenores_20210106_030028....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012_...                     0.0   
PlataformasAgregadasSinMenores_20210109_030014....                     0.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211229_030012....                     1.0   
PlataformasAgregadasSinMenores_20211229_030012_...                     1.0   
PlataformasAgregadasSinMenores_20211230_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012_...                     1.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20210105_030039....     0.000000  
PlataformasAgregadasSinMenores_20210106_030028....     0.000000  
PlataformasAgregadasSinMenores_20210108_030012....     0.000000  
PlataformasAgregadasSinMenores_20210108_030012_...     0.000000  
PlataformasAgregadasSinMenores_20210109_030014....     0.000000  
..                                                          ...  
PlataformasAgregadasSinMenores_20211229_030012....     0.600000  
PlataformasAgregadasSinMenores_20211229_030012_...     0.471698  
PlataformasAgregadasSinMenores_20211230_030012....     0.534884  
PlataformasAgregadasSinMenores_20211231_030012....     0.582645  
PlataformasAgregadasSinMenores_20211231_030012_...     0.583333  

[414 rows x 2 columns]

##### Entity

Files (given by the index) in which *GenCat* has not a single URL filled in, neither have the rest

In [ ]:
only_last_update_df.loc[filename_notna_ratio.index[filename_notna_ratio[url_entity_col] == 0]][url_entity_col].isna().all()

True

Files in which *GenCat* has *some* but not all URLs filled in

In [ ]:
filename_notna_ratio[(filename_notna_ratio[url_entity_col] > 0) & (filename_notna_ratio[url_entity_col] < 1.)]

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20211020_030012_...                     0.5   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20211020_030012_...       0.4375

##### Winner

Maximum coverage

In [ ]:
filename_notna_ratio[winner_col].max()

0.8928571428571429

Unlike for the *Buyer* field above, files (given by the index) in which *GenCat* has not a single *Winner* filled in have entries filled in by other organizations

In [ ]:
only_last_update_df.loc[filename_notna_ratio.index[filename_notna_ratio[winner_col] == 0]][winner_col].isna().all()

False

Files in which *GenCat* has 0 coverage for *Winner*

In [ ]:
gencat_winner_zero_coverage = filename_notna_ratio[filename_notna_ratio[winner_col] == 0]
gencat_winner_zero_coverage

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20210105_030039....                     0.0   
PlataformasAgregadasSinMenores_20210106_030028....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012....                     0.0   
PlataformasAgregadasSinMenores_20210108_030012_...                     0.0   
PlataformasAgregadasSinMenores_20210109_030014....                     0.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211015_030012_...                     0.0   
PlataformasAgregadasSinMenores_20211016_030014....                     0.0   
PlataformasAgregadasSinMenores_20211016_030014_...                     0.0   
PlataformasAgregadasSinMenores_20211019_030015....                     0.0   
PlataformasAgregadasSinMenores_20211019_030015_...                     0.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20210105_030039....          0.0  
PlataformasAgregadasSinMenores_20210106_030028....          0.0  
PlataformasAgregadasSinMenores_20210108_030012....          0.0  
PlataformasAgregadasSinMenores_20210108_030012_...          0.0  
PlataformasAgregadasSinMenores_20210109_030014....          0.0  
..                                                          ...  
PlataformasAgregadasSinMenores_20211015_030012_...          0.0  
PlataformasAgregadasSinMenores_20211016_030014....          0.0  
PlataformasAgregadasSinMenores_20211016_030014_...          0.0  
PlataformasAgregadasSinMenores_20211019_030015....          0.0  
PlataformasAgregadasSinMenores_20211019_030015_...          0.0  

[318 rows x 2 columns]

Indeed...

In [ ]:
assert gencat_df.loc[gencat_winner_zero_coverage.index, winner_col].isna().all()

What about overall (not only *GenCat*)?

In [ ]:
only_last_update_df.loc[gencat_winner_zero_coverage.index, winner_col].isna().all()

False

An example of file with zero coverage

In [ ]:
gencat_df.loc['PlataformasAgregadasSinMenores_20210105_030039.atom', id_col].tolist()

['https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6578285',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6842544',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6745938',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6770807']

A couple of example files

In [ ]:
gencat_df.loc[gencat_winner_zero_coverage.index[:2]]

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    https://contrataciondelestado.es/sindicacion/P...   
                                                   343    https://contrataciondelestado.es/sindicacion/P...   
                                                   342    https://contrataciondelestado.es/sindicacion/P...   
                                                   173    https://contrataciondelestado.es/sindicacion/P...   
PlataformasAgregadasSinMenores_20210106_030028.... 465    https://contrataciondelestado.es/sindicacion/P...   
                                                   60     https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Id licitación: 001/20001709; Órgano de Contrat...   
                                                   343    Id licitación: X2020025214; Órgano de Contrata...   
                                                   342    Id licitación: 7/2020/COSSE; Órgano de Contrat...   
                                                   173    Id licitación: 2020-358/1403; Órgano de Contra...   
PlataformasAgregadasSinMenores_20210106_030028.... 465    Id licitación: 20004065; Órgano de Contratació...   
                                                   60     Id licitación: 1215/2020; Órgano de Contrataci...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20210105_030039.... 352    Serveis de Manteniment i subministrament de le...

In [ ]:
# outsiders_df.loc[filename_notna_ratio[filename_notna_ratio[winner_col] == 0].index, winner_col].isna()

Files in which *GenCat* has *some* but not all *Winner*s filled in

In [ ]:
filename_notna_ratio[(filename_notna_ratio[winner_col] > 0) & (filename_notna_ratio[winner_col] < 1.)]

ContractFolderStatus  \
                                                   LocatedContractingParty   
                                                         BuyerProfileURIID   
                                                                             
                                                                             
                                                                             
                                                                             
file name                                                                    
PlataformasAgregadasSinMenores_20211020_030012....                     1.0   
PlataformasAgregadasSinMenores_20211020_030012_...                     0.5   
PlataformasAgregadasSinMenores_20211021_030015....                     1.0   
PlataformasAgregadasSinMenores_20211021_030015_...                     1.0   
PlataformasAgregadasSinMenores_20211022_030030....                     1.0   
..                                                                     ...   
PlataformasAgregadasSinMenores_20211229_030012....                     1.0   
PlataformasAgregadasSinMenores_20211229_030012_...                     1.0   
PlataformasAgregadasSinMenores_20211230_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012....                     1.0   
PlataformasAgregadasSinMenores_20211231_030012_...                     1.0   

                                                                 
                                                   TenderResult  
                                                   WinningParty  
                                                      PartyName  
                                                           Name  
                                                                 
                                                                 
file name                                                        
PlataformasAgregadasSinMenores_20211020_030012....     0.892857  
PlataformasAgregadasSinMenores_20211020_030012_...     0.437500  
PlataformasAgregadasSinMenores_20211021_030015....     0.736264  
PlataformasAgregadasSinMenores_20211021_030015_...     0.673469  
PlataformasAgregadasSinMenores_20211022_030030....     0.824176  
..                                                          ...  
PlataformasAgregadasSinMenores_20211229_030012....     0.600000  
PlataformasAgregadasSinMenores_20211229_030012_...     0.471698  
PlataformasAgregadasSinMenores_20211230_030012....     0.534884  
PlataformasAgregadasSinMenores_20211231_030012....     0.582645  
PlataformasAgregadasSinMenores_20211231_030012_...     0.583333  

[96 rows x 2 columns]

---

#### Xunta de Galicia

Entries for *Xunta de Galicia*

In [ ]:
gal_df = only_last_update_df[only_last_update_df['domain'] == 'www.contratosdegalicia.gal']
gal_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80     https://contrataciondelestado.es/sindicacion/P...   
PlataformasAgregadasSinMenores_20211020_030012.... 466    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80     Id Licitación: PcPG/2021/802999, Órgano de Con...   
PlataformasAgregadasSinMenores_20211020_030012.... 466    Id Licitación: PcPG/2019/616022, Órgano de Con...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20211020_030012_... 80                   Saneamiento en camino de la Carousa   
PlataformasAgregadasSinMenores_20211020_030012.... 466    Proyecto de ahorro y eficencia energética en e...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

*Xunta de Galicia* **always** provides the URL

In [ ]:
gal_df[url_entity_col].notna().all()

True

### Naming

A mapping between columns and *human-readable* fields can be found [here](https://docs.google.com/spreadsheets/d/1vRN_bLWyUp6Hs1ONUSP9m3cfrgjx5bfbD4wzW4gXCu0/edit#gid=0). The latter was processed in `naming.ipynb` to get the file below.

In [ ]:
input_file = pathlib.Path.cwd() / 'PLACE.yaml'
assert input_file.exists()

It provides (as a `dict`) a mapping from *human-readable* names to (maybe nested) fields in *Atom* files

In [ ]:
with open(input_file) as yaml_data:
    data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)
{k: data_scheme[k] for k in list(data_scheme)[:5]}

{'id': ['id', nan, nan, nan, nan, nan, nan],
 'summary': ['summary', nan, nan, nan, nan, nan, nan],
 'title': ['title', nan, nan, nan, nan, nan, nan],
 'updated': ['updated', nan, nan, nan, nan, nan, nan],
 'Número de Expediente': ['ContractFolderStatus',
  'ContractFolderID',
  nan,
  nan,
  nan,
  nan,
  nan]}

The mapping can be exploited to access the columns of the `pd.DataFrame` using more natural names

In [ ]:
outsiders_df[dlsproc.hier.pad_col_levels(outsiders_df, data_scheme['Número de Expediente'], denan=True)]

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                       3064_23/2021
                                                       1        3051_CONT_SERVICIOS/2021/40
                                                       2                       3091_21/2021
                                                       3                        3069_1/2021
                                                       4                    8044_30146-0-21
                                                                           ...             
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                          2021.63
                                                       464                          2021.67
                                                       465                     C02/015/2021
                                                       466                    EJIE-146-2021
                   

Columns of the `pd.DataFrame` are renamed with *plain* names (`pd.MultiIndex` is gone)

In [ ]:
# stateful_outsiders_df = dlsproc.assemble.flatten_columns_names(stateful_outsiders_df, data_scheme)

A convenience function to go from *human-readable* name to actual *multiindex* column

In [ ]:
n2c = lambda n: dlsproc.hier.pad_col_levels(outsiders_df, data_scheme[n], denan=True)

The column belows, e.g., mixes `list`s and elemental values

In [ ]:
# outsiders_df['Clasificación CPV']
outsiders_df[n2c('Clasificación CPV')]

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                                        77310000
                                                       1                                        71241000
                                                       2                                        45231400
                                                       3        [45231300, 45213260, 45112000, 44611600]
                                                       4                                        45200000
                                                                                  ...                   
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                                      45210000
                                                       464                                      22200000
                                                       465                                      805

### Saving

In [ ]:
stateful_outsiders_df.loc['PlataformasAgregadasSinMenores_20210101_030039.atom'].head(2)

entry  \
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
id                                                         
https://contrataciondelestado.es/sindicacion/Pl...   294   
https://contrataciondelestado.es/sindicacion/Pl...   291   

                                                                                              summary  \
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
id                                                                                                      
https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   

                                                                                                title  \
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
id                                                                                                      
https://contrataciondelestado.es/sindicacion/Pl...  SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
https://contrataciondelestado.es/sindicacion/Pl...  Subministrament d'energia elèctrica en BAIXA T...   

                                                                            updated  \
                                                                                      
                                                                                      
                                                                                      
                                                                                      
                                                                                      
                                                                                      
id                                                                                    
https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:25:27.661000+00:00   
https://contrataciondelestado.es/sindicacion/Pl... 2020-12-30 14:26:25.241000+00:00   

                                                   ContractFolderStatus  \
                                                       ContractFolderID   
                                                                          
                                                                          
                                                                          
                                                                          
                           

In saving to *parquet* one must be careful (see [this post](http://www.legendu.net/misc/blog/python-complicated-data-types/))

In [ ]:
final_df = dlsproc.assemble.parquet_amenable(stateful_outsiders_df)
final_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...   294   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   291   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                          

Metadata (right now, it seems it's not saved when writing the `pd.DataFrame` to disk)

In [ ]:
# stateful_outsiders_df.attrs['# duplicate deletes'] = n_duplicate_deletes

Saving to *parquet* (and maybe in the [Google Drive shared unit](https://drive.google.com/drive/folders/0ADw_MTEsA4fSUk9PVA))

In [ ]:
output_file = outsiders_output_file.name.with_suffix('.parquet')
final_df.to_parquet(output_file)

In [ ]:
restored_df = pd.read_parquet(output_file)
restored_df.head()

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...   294   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   291   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   288   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   282   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   281   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/168; Órgano de Contrat...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 5/2020; Órgano de Contratación:...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/169; Órgano de Contrat...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
     

In [ ]:
len(restored_df)

64020

In [ ]:
dlsproc.assemble.flatten_columns_names(restored_df, data_scheme)

entry  \
file name                                          id                                                          
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...    294   
                                                   https://contrataciondelestado.es/sindicacion/Pl...    291   
                                                   https://contrataciondelestado.es/sindicacion/Pl...    288   
                                                   https://contrataciondelestado.es/sindicacion/Pl...    282   
                                                   https://contrataciondelestado.es/sindicacion/Pl...    281   
...                                                                                                      ...   
PlataformasAgregadasSinMenores.atom                https://contrataciondelestado.es/sindicacion/Pl...      4   
                                                   https://contrataciondelestado.es/sindicacion/Pl...      3   
                                                   https://contrataciondelestado.es/sindicacion/Pl...      2   
                                                   https://contrataciondelestado.es/sindicacion/Pl...      1   
                                                   https://contrataciondelestado.es/sindicacion/Pl...      0   

                                                                                                                                                 summary  \
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/168; Órgano de Contrat...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 5/2020; Órgano de Contratación:...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PRO/2020/169; Órgano de Contrat...   
...                                                                                                                                                  ...   
PlataformasAgregadasSinMenores.atom                https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 8044_30146-0-21; Órgano de Cont...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 3069_1/2021; Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 3091_21/2021; Órgano de Contrat...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 3051_CONT_SERVICIOS/2021/40; Ór...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 3064_23/2021; Órgano de Contrat...   

                                                                                                                                                   title  \
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Subministrament d'energia elèctrica en BAIXA T

## 2019

A file to *cache* the data read

In [ ]:
outsiders_2019_file = dlsproc.io.File('outsiders_2019')

In [ ]:
outsiders_2019_zip_file = outsiders_directory / 'PlataformasAgregadasSinMenores_2019.zip'
assert outsiders_2019_zip_file.exists()

The data is read from a file if present, or from the given **zip** if not

In [ ]:
if outsiders_2019_file.exists():
    outsiders_2019_df = outsiders_2019_file.read()
else:
    # data is read from the given zip, and concatenated into a single `pd.DataFrame`...
    outsiders_2019_df = dlsproc.bundle.read_zip(outsiders_2019_zip_file, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_2019_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_2019_df)
    
    # ...and saved
    outsiders_2019_file.write(outsiders_2019_df)

In [ ]:
df = outsiders_2019_df

In [ ]:
deduplicated_df = dlsproc.postprocess.keep_updates_only(df)
deduplicated_df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20190103_031259.... 292    https://contrataciondelestado.es/sindicacion/P...   
                                                   289    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20190103_031259.... 292    Id licitación: 34/18; Órgano de Contratación: ...   
                                                   289    Id licitación: G/130/20/4/1539/O621/0000/11201...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20190103_031259.... 292    Execució de les obres i instal·lacions relativ...   
                                                   289    Suministro de campímetro para el Hospital Univ...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenor

In [ ]:
deleted_series = dlsproc.bundle.read_deleted_zip(outsiders_2019_zip_file)
deleted_series

file name                                              id                                                                                 
PlataformasAgregadasSinMenores.atom                    https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/3493892    2020-01-01 02:11:21.072000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/3266333    2020-01-01 00:10:58.328000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/3336074    2020-01-01 00:10:58.212000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/3253172    2020-01-01 00:10:58.096000+01:00
                                                       https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/3266309    2020-01-01 00:10:57

In [ ]:
stateful_df = dlsproc.assemble.merge_deleted(deduplicated_df, deleted_series)
stateful_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...   292   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   289   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 34/18; Órgano de Contratación: ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: G/130/20/4/1539/O621/0000/11201...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Execució de les obres i instal·lacions relativ...   
                          

In [ ]:
stateful_df['deleted_on'].notna().sum()

13

In [ ]:
stateful_df.shape

(60024, 41)

In [ ]:
to_be_saved_df = dlsproc.assemble.parquet_amenable(stateful_df)
to_be_saved_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...   292   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   289   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 34/18; Órgano de Contratación: ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: G/130/20/4/1539/O621/0000/11201...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Execució de les obres i instal·lacions relativ...   
                          

## Merging

In [ ]:
stateful_outsiders_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...   294   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   291   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: PA-02-2020;  Órgano de Contrata...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2006OB07; Órgano de Contratació...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20210101_030039.... https://contrataciondelestado.es/sindicacion/Pl...  SERVICIO DE RECEPCIÓN Y ATENCIÓN TELEFÓNICA A ...   
                          

In [ ]:
stateful_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...   292   
                                                   https://contrataciondelestado.es/sindicacion/Pl...   289   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 34/18; Órgano de Contratación: ...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: G/130/20/4/1539/O621/0000/11201...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...  Execució de les obres i instal·lacions relativ...   
                          

Number of rows for one and the other

In [ ]:
len(stateful_outsiders_df), len(stateful_df)

(64020, 60024)

Same number of columns

In [ ]:
assert stateful_outsiders_df.columns.shape == stateful_df.columns.shape

Same number of levels

In [ ]:
assert stateful_outsiders_df.columns.nlevels == stateful_df.columns.nlevels

In [ ]:
stateful_outsiders_df.columns == stateful_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False,  True])

In [ ]:
set(stateful_outsiders_df.columns) == set(stateful_df.columns)

False

Columns only in the first

In [ ]:
set(stateful_outsiders_df.columns) - set(stateful_df.columns)

{('ContractFolderStatus',
  'LocatedContractingParty',
  'BuyerProfileURIID',
  '',
  '',
  '',
  '')}

Columns only in the second

In [ ]:
set(stateful_df.columns) - set(stateful_outsiders_df.columns)

{('ContractFolderStatus',
  'TenderingProcess',
  'TenderSubmissionDeadlinePeriod',
  'Description',
  '',
  '',
  '')}

In [ ]:
res = pd.concat((stateful_df, stateful_outsiders_df), axis=0, keys=[outsiders_2019_zip_file.name, outsiders_zip_file.name], names=['zip', 'file name', 'id'])
res

entry  \
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
                                                                                                                                                      
zip                                     file name                                          id                                                         
PlataformasAgregadasSinMenores_2019.zip PlataformasAgregadasSinMenores_20190103_031259.... https://contrataciondelestado.es/sindicacion/Pl...   292   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...   289   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...   288   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...   287   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...   285   
..                                                                                                                                              ...   
PlataformasAgregadasSinMenores_2021.zip PlataformasAgregadasSinMenores.atom                https://contrataciondelestado.es/sindicacion/Pl...     4   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...     3   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...     2   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...     1   
                                                                                           https://contrataciondelestado.es/sindicacion/Pl...     0   

                                                                                                                                                                                         summary  \
                                                                                                                                                                                                   
                                                                                                                                                                                                   
                                                                                                                                                                                                   
                                                                                                                                                                                                   
                                                                                                                                                                                                   
                                                                                                